In [1]:
import numpy as np
import pandas as pd
# from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment

In [2]:
def what_is_installed():
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except ModuleNotFoundError:
    !pip install pycaret
    what_is_installed()


System:
    python: 3.10.13 (main, Sep 11 2023, 08:39:02) [Clang 14.0.6 ]
executable: /Users/devinhopkins/anaconda3/envs/ed-flow-2023/bin/python
   machine: macOS-10.16-x86_64-i386-64bit

PyCaret required dependencies:
                 pip: 23.3
          setuptools: 68.0.0
             pycaret: 3.2.0
             IPython: 8.17.2
          ipywidgets: 8.1.1
                tqdm: 4.66.1
               numpy: 1.25.2
              pandas: 1.5.3
              jinja2: 3.1.2
               scipy: 1.10.1
              joblib: 1.3.2
             sklearn: 1.2.2
                pyod: 1.1.2
            imblearn: 0.11.0
   category_encoders: 2.6.3
            lightgbm: 4.1.0
               numba: 0.58.1
            requests: 2.31.0
          matplotlib: 3.6.0
          scikitplot: 0.3.7
         yellowbrick: 1.5
              plotly: 5.18.0
    plotly-resampler: Not installed
             kaleido: 0.2.1
           schemdraw: 0.15
         statsmodels: 0.14.0
              sktime: 0.21.1
         

In [3]:
# Global Figure Settings for notebook ----
# Depending on whether you are using jupyter notebook, jupyter lab, Google Colab, you may have to set the renderer appropriately
# NOTE: Setting to a static renderer here so that the notebook saved size is reduced.
global_fig_settings = {
    # "renderer": "notebook",
    "renderer": "png",
    "width": 1000,
    "height": 600,
}

In [5]:
data = pd.read_csv('allData.csv')
data.ds = pd.to_datetime(data.ds)
data

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,AMBVERT1,AMBVERTTBS,QTrack_TBS,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM
0,2021-01-01 01:00:00,1,1,1,1,2,2,0,0,0,...,0,0,NaN,NaN,1,0,0,0,3,3
1,2021-01-01 02:00:00,2,3,1,2,3,5,1,1,0,...,2,2,NaN,NaN,1,1,0,0,3,3
2,2021-01-01 03:00:00,0,3,1,3,1,6,0,1,0,...,2,1,NaN,NaN,0,0,0,0,3,3
3,2021-01-01 04:00:00,1,4,0,3,1,7,1,2,0,...,2,0,NaN,NaN,0,0,0,0,4,3
4,2021-01-01 05:00:00,1,5,1,4,2,9,1,3,0,...,1,1,NaN,NaN,0,0,0,0,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25098,2023-11-14 10:00:00,9,48,5,23,14,71,1,11,0,...,14,10,2.0,0.0,14,0,0,4,11,6
25099,2023-11-14 11:00:00,12,60,10,33,22,93,5,16,0,...,17,12,5.0,0.0,16,0,0,7,10,6
25100,2023-11-14 12:00:00,16,76,9,42,25,118,3,19,0,...,21,16,7.0,0.0,12,0,0,4,10,5
25101,2023-11-14 13:00:00,14,90,7,49,21,139,1,20,0,...,28,24,5.0,0.0,12,0,0,5,10,5


In [12]:
data = data.drop_duplicates(subset='ds', keep='first')
print(len(data.ds.unique()))
print(len(data.ds))

25100
25100


In [18]:
data.isna().sum()

ds                        0
INFLOW_STRETCHER          0
Infl_Stretcher_cum        0
INFLOW_AMBULATORY         0
Infl_Ambulatory_cum       0
Inflow_Total              0
Inflow_Cum_Total          0
INFLOW_AMBULANCES         0
Infl_Ambulances_cum       0
FLS                       0
CUM_ADMREQ                0
CUM_BA1                   0
WAITINGADM                0
TTStr                     0
TRG_HALLWAY1              0
TRG_HALLWAY_TBS           0
reoriented_cum            0
reoriented_cum_MD         0
QTRACK1                   0
RESUS                     0
Pod_T                     0
POD_GREEN                 0
POD_GREEN_TBS             0
POD_YELLOW                0
POD_YELLOW_TBS            0
POD_ORANGE                0
POD_ORANGE_TBS            0
POD_CONS_MORE2H           0
POD_IMCONS_MORE4H         0
POD_XRAY_MORE2H           0
POD_CT_MORE2H             0
POST_POD1                 0
VERTSTRET                 0
RAZ_TBS                   0
RAZ_LAZYBOY               0
RAZ_WAITINGREZ      

In [20]:
cleaned.isna().sum()

INFLOW_STRETCHER         42
Infl_Stretcher_cum       42
INFLOW_AMBULATORY        42
Infl_Ambulatory_cum      42
Inflow_Total             42
Inflow_Cum_Total         42
INFLOW_AMBULANCES        42
Infl_Ambulances_cum      42
FLS                      42
CUM_ADMREQ               42
CUM_BA1                  42
WAITINGADM               42
TTStr                    42
TRG_HALLWAY1             42
TRG_HALLWAY_TBS          42
reoriented_cum           42
reoriented_cum_MD        42
QTRACK1                  42
RESUS                    42
Pod_T                    42
POD_GREEN                42
POD_GREEN_TBS            42
POD_YELLOW               42
POD_YELLOW_TBS           42
POD_ORANGE               42
POD_ORANGE_TBS           42
POD_CONS_MORE2H          42
POD_IMCONS_MORE4H        42
POD_XRAY_MORE2H          42
POD_CT_MORE2H            42
POST_POD1                42
VERTSTRET                42
RAZ_TBS                  42
RAZ_LAZYBOY              42
RAZ_WAITINGREZ           42
AMBVERT1            

25100
25100


In [8]:
from pycaret.utils.time_series import clean_time_index
cleaned = clean_time_index(data=data, index_col='ds', freq="H")

In [9]:
# Create an EDA experiment ----
exp = TSForecastingExperiment()

In [23]:
exp.setup(
    data=cleaned[['POST_POD1']],
    target='POST_POD1',
    # index=index,
    fh=24,
    numeric_imputation_target="ffill",
    numeric_imputation_exogenous="ffill",
    # Set defaults for the plots ----
    fig_kwargs=global_fig_settings,
    session_id=42,
)

,Description,Value
0,session_id,42
1,Target,POST_POD1
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(25142, 1)"
5,Transformed data shape,"(25142, 1)"
6,Transformed train set shape,"(25118, 1)"
7,Transformed test set shape,"(24, 1)"
8,Rows with missing values,0.2%
9,Fold Generator,ExpandingWindowSplitter


In [24]:
best = exp.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:22:12
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Naive Forecaster


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
naive,Naive Forecaster,0.9510,0.6071,1.1806,1.5221,0.2926,0.2182,-0.6431,1.2067
arima,ARIMA,1.1043,0.6573,1.3709,1.6481,0.2676,0.2707,-1.6697,3.3367
snaive,Seasonal Naive Forecaster,1.4546,0.9230,1.8056,2.3139,0.3327,0.4562,-3.3166,0.5600
polytrend,Polynomial Trend Forecaster,1.7588,0.9714,2.1836,2.4357,0.3709,0.4657,-4.9049,0.5233
grand_means,Grand Means Forecaster,3.3207,1.7137,4.1228,4.2968,0.7332,1.1700,-16.8921,0.5500


Processing:   0%|          | 0/113 [00:00<?, ?it/s]

KeyboardInterrupt: 